In [3]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/NLP_Assignment/')

Mounted at /content/drive


In [4]:
import os

# Set the CLASSPATH environment variable
os.environ['CLASSPATH'] = '/content/drive/MyDrive/surya/NLP_Assignment/stanford-ner.jar'


In [5]:
import os
import zipfile
from urllib.request import urlretrieve
from nltk.tag import StanfordNERTagger

# Set the path for the Stanford NER model
stanford_ner_model_path = '/content/drive/MyDrive/surya/NLP_Assignment/english.all.3class.distsim.crf.ser.gz'

# Check if the model file exists, if not, download it
if not os.path.exists(stanford_ner_model_path):
    print("Downloading Stanford NER model...")
    stanford_ner_model_url = 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'  # Update with the correct URL
    zip_file_path, _ = urlretrieve(stanford_ner_model_url)

    # Extract the downloaded zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('/content/drive/MyDrive/surya/NLP_Assignment/')

    # Remove the downloaded zip file
    os.remove(zip_file_path)

# Set the STANFORD_MODELS environment variable
os.environ['STANFORD_MODELS'] = '/content/drive/MyDrive/surya/NLP_Assignment/'

# Load the StanfordNER model
stanford_ner = StanfordNERTagger(
    '/content/drive/MyDrive/surya/NLP_Assignment/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
    '/content/drive/MyDrive/surya/NLP_Assignment/stanford-ner-2018-10-16/stanford-ner.jar'
)

# Continue with the rest of your code...


In [19]:
import csv

# Load the NER dataset using csv.reader
with open('NER_Dataset.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)  # Skip the header
    dataset = list(reader)

# Perform text processing and evaluation using SpaCy
spacy_predictions = []
spacy_ground_truth = []
for row in dataset:
    if len(row) >= 4:
        sentence_id, words, pos_tags, ner_tags = row
        words_list = eval(words)  # Convert the string representation of the list to a list
        tokens, pos_tags, ner_tags = process_with_spacy(" ".join(words_list))
        spacy_predictions.extend(ner_tags)
        spacy_ground_truth.extend(tokens)

# Evaluate model predictions
spacy_accuracy, spacy_confusion_matrix = evaluate_predictions(spacy_predictions, spacy_ground_truth)

# Print results
print("SpaCy Accuracy:", spacy_accuracy)
print("Confusion Matrix:", spacy_confusion_matrix)


SpaCy Accuracy: 0.0
Confusion Matrix: {'Thousands': {None: 6}, 'of': {None: 477}, 'demonstrators': {None: 6}, 'have': {None: 81}, 'marched': {None: 1}, 'through': {None: 6}, 'London': {None: 8}, 'to': {None: 417}, 'protest': {None: 6}, 'the': {None: 896}, 'war': {None: 15}, 'in': {None: 461}, 'Iraq': {None: 27}, 'and': {None: 393}, 'demand': {None: 1}, 'withdrawal': {None: 6}, 'British': {None: 18}, 'troops': {None: 31}, 'from': {None: 71}, 'that': {None: 105}, 'country': {None: 45}, '.': {None: 824}, 'Iranian': {None: 9}, 'officials': {None: 39}, 'say': {None: 59}, 'they': {None: 34}, 'expect': {None: 3}, 'get': {None: 3}, 'access': {None: 8}, 'sealed': {None: 1}, 'sensitive': {None: 1}, 'parts': {None: 2}, 'plant': {None: 1}, 'Wednesday': {None: 20}, ',': {None: 570}, 'after': {None: 41}, 'an': {None: 74}, 'IAEA': {None: 2}, 'surveillance': {None: 1}, 'system': {None: 7}, 'begins': {None: 2}, 'functioning': {None: 1}, 'Helicopter': {None: 1}, 'gunships': {None: 2}, 'Saturday': {None:

In [6]:
import spacy
from nltk import pos_tag
from nltk.tag import StanfordNERTagger

# Load the NER dataset
dataset = open('NER_Dataset.csv', 'r').readlines()

# Load the SpaCy model
nlp = spacy.load('en_core_web_sm')

# Load the StanfordNER model
#stanford_ner = StanfordNERTagger('/content/drive/MyDrive/surya/NLP_Assignment/english.all.3class.distsim.crf.ser.gz')

# Define the TAG mapping
tag_mapping = {
    "B-art": "ART",
    "B-eve": "EVENT",
    "B-geo": "GPE",
    "B-gpe": "GPE",
    "B-nat": "NORP",
    "B-org": "ORG",
    "B-per": "PERSON",
    "B-tim": "TIME",
    "I-art": "ART",
    "I-eve": "EVENT",
    "I-geo": "GPE",
    "I-gpe": "GPE",
    "I-nat": "NORP",
    "I-org": "ORG",
    "I-per": "PERSON",
    "I-tim": "TIME",
    "O": ""
}

# Perform text processing using SpaCy
def process_with_spacy(sentence):
    doc = nlp(sentence)
    tokens = [token.text for token in doc]
    pos_tags = [token.pos_ for token in doc]
    ner_tags = [tag_mapping.get(ent.label_) for ent in doc.ents]
    return tokens, pos_tags, ner_tags

# Perform text processing using NLTK
def process_with_nltk(sentence):
    tokens = sentence.split()
    pos_tags = [tag for _, tag in pos_tag(tokens)]
    ner_tags = stanford_ner.tag(tokens)
    ner_tags = [tag_mapping.get(tag) for _, tag in ner_tags]
    return tokens, pos_tags, ner_tags

# Evaluate model predictions
def evaluate_predictions(predictions, ground_truth):
    # Calculate accuracy
    accuracy = sum(1 for pred, gt in zip(predictions, ground_truth) if pred == gt) / len(predictions)

    # Calculate confusion matrix
    confusion_matrix = {}
    for pred, gt in zip(predictions, ground_truth):
        if gt not in confusion_matrix:
            confusion_matrix[gt] = {}
        if pred not in confusion_matrix[gt]:
            confusion_matrix[gt][pred] = 0
        confusion_matrix[gt][pred] += 1

    return accuracy, confusion_matrix

# Perform text processing and evaluation using SpaCy
spacy_predictions = []
spacy_ground_truth = []
for line in dataset:
    sentence_id, sentence = line.strip().split('\t')
    tokens, pos_tags, ner_tags = process_with_spacy(sentence)
    spacy_predictions.extend(ner_tags)
    spacy_ground_truth.extend(tokens)

spacy_accuracy, spacy_confusion_matrix = evaluate_predictions(spacy_predictions, spacy_ground_truth)

# Perform text processing and evaluation using NLTK
nltk_predictions = []
nltk_ground_truth = []
for line in dataset:
    sentence_id, sentence = line.strip().split('\t')
    tokens, pos_tags, ner_tags = process_with_nltk(sentence)
    nltk_predictions.extend(ner_tags)
    nltk_ground_truth.extend(tokens)

nltk_accuracy, nltk_confusion_matrix = evaluate_predictions(nltk_predictions, nltk_ground_truth)

# Perform text processing and evaluation using StanfordNER
stanford_predictions = []
stanford_ground_truth = []
for line in dataset:
    sentence_id, sentence = line.strip().split('\t')
    tokens, pos_tags, ner_tags = process_with_nltk(sentence)
    stanford_predictions.extend(ner_tags)
    stanford_ground_truth.extend(tokens)

stanford_accuracy, stanford_confusion_matrix = evaluate_predictions(stanford_predictions, stanford_ground_truth)

# Print the results
print("SpaCy Accuracy:", spacy_accuracy)
print("SpaCy Confusion Matrix:", spacy_confusion_matrix)
print("NLTK Accuracy:", nltk_accuracy)
print("NLTK Confusion Matrix:", nltk_confusion_matrix)
print("StanfordNER Accuracy:", stanford_accuracy)
print("StanfordNER Confusion Matrix:", stanford_confusion_matrix)


ValueError: not enough values to unpack (expected 2, got 1)